# Naive Bayes - Trabalho

## Questão 1

Implemente um classifacor Naive Bayes para o problema de predizer a qualidade de um carro. Para este fim, utilizaremos um conjunto de dados referente a qualidade de carros, disponível no [UCI](https://archive.ics.uci.edu/ml/datasets/car+evaluation). Este dataset de carros possui as seguintes features e classe:

** Attributos **
1. buying: vhigh, high, med, low
2. maint: vhigh, high, med, low
3. doors: 2, 3, 4, 5, more
4. persons: 2, 4, more
5. lug_boot: small, med, big
6. safety: low, med, high

** Classes **
1. unacc, acc, good, vgood




In [24]:
import csv
import math

In [25]:
 def loadCsv(filename):
    lines = csv.reader(open(filename, "r"))
    dataset = list(lines)
    for i in range(len(dataset)):
        dataset[i] = [str(x) for x in dataset[i]]
    return dataset

import random
def splitDataset(dataset, splitRatio):
    trainSize = int(len(dataset) * splitRatio)
    trainSet = []
    copy = list(dataset)
    while len(trainSet) < trainSize:
        index = random.randrange(len(copy))
        trainSet.append(copy.pop(index))
    return [trainSet, copy]

def separateByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated


def mean(numbers):
    return sum(numbers)/float(len(numbers))
 
def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)

def summarize(dataset):
    summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
    del summaries[-1]
    return summaries

def summarizeByClass(dataset):
    separated = separateByClass(dataset)
    summaries = {}
    for classValue, instances in separated.items():
        summaries[classValue] = summarize(instances)
    return summaries

def calculateProbability(x, mean, stdev):
    if (stdev == 0.0):
        exponent = 0.0
        retorno = 0.0
    else:
        exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
        retorno = (1 / (math.sqrt(2*math.pi) * stdev)) * exponent
    
    return retorno

def calculateClassProbabilities(summaries, inputVector):
    probabilities = {}
    for classValue, classSummaries in summaries.items():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean, stdev = classSummaries[i]
            x = inputVector[i]
            probabilities[classValue] *= calculateProbability(x, mean, stdev)
    return probabilities

def predict(summaries, inputVector):
    probabilities = calculateClassProbabilities(summaries, inputVector)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel

def getPredictions(summaries, testSet):
    predictions = []
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predictions.append(result)
    return predictions

def getAccuracy(testSet, predictions):
    correct = 0
    for i in range(len(testSet)):
        if testSet[i][-1] == predictions[i]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

In [26]:
ds = loadCsv('carData.csv')

## Questão 2
Crie uma versão de sua implementação usando as funções disponíveis na biblioteca SciKitLearn para o Naive Bayes ([veja aqui](http://scikit-learn.org/stable/modules/naive_bayes.html)) 



In [27]:
from sklearn.naive_bayes import GaussianNB
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [28]:
df = pd.read_csv('carData.csv', names=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'classe'])

df['safety'] = LabelEncoder().fit(df.safety).transform(df.safety)
df['classe'] = LabelEncoder().fit(df.classe).transform(df.classe)
df['doors'] = LabelEncoder().fit(df.doors).transform(df.doors)
df['persons'] = LabelEncoder().fit(df.persons).transform(df.persons)
df['buying'] = LabelEncoder().fit(df.buying).transform(df.buying)
df['maint'] = LabelEncoder().fit(df.maint).transform(df.maint)
df['lug_boot'] = LabelEncoder().fit(df.lug_boot).transform(df.lug_boot)


df.head()

,buying,maint,doors,persons,lug_boot,safety,classe
0,3,3,0,0,2,1,2
1,3,3,0,0,2,2,2
2,3,3,0,0,2,0,2
3,3,3,0,0,1,1,2
4,3,3,0,0,1,2,2


In [29]:
gnb = GaussianNB()

In [30]:
features = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']
X = df.get(features)
Y = df['classe'].values
X = MinMaxScaler().fit_transform(df[features])

In [31]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state=1)
y_pred = gnb.fit(X_train, Y_train).predict(X_test)
accuracy_score(Y_test, y_pred)

0.65703275529865124

In [32]:
print(classification_report(y_pred, Y_test))

             precision    recall  f1-score   support

          0       0.12      0.68      0.20        19
          1       0.00      0.00      0.00         0
          2       0.84      0.84      0.84       368
          3       1.00      0.14      0.24       132

avg / total       0.86      0.66      0.67       519



/home/leozin/miniconda3/envs/ambiente/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
